In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Input
from tensorflow.keras.layers import Reshape, MaxPooling2D
from tensorflow.keras.layers import Conv2D, Dense, Flatten
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import os

2024-05-11 16:13:15.217915: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-11 16:13:15.237495: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-11 16:13:15.237514: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-11 16:13:15.237935: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-11 16:13:15.241202: I tensorflow/core/platform/cpu_feature_guar

In [2]:
# Check tensorflow version
tf.__version__

'2.15.0'

In [3]:
train_marker_path = '../data/train/marker_train_2.jpeg'
label_file_path = '../data/train/image_descriptions.txt'

In [4]:
# Define a directory for saved files
output_dir = '../data/train'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [5]:
# Constants
img_shape_full = (300, 300, 1)
num_classes = 2

In [6]:
# Get a list of all .npy files in the directory
npy_files = [f for f in os.listdir(output_dir) if f.endswith('.npy')]

In [7]:
# Read labels from the text file
labels = {}
with open(label_file_path, 'r') as file:
    for line in file:
        filename, label = line.strip().split(',')
        labels[filename] = int(label)

# Load images and their labels based on the filenames
images = []
image_labels = []
for npy_file in sorted(os.listdir(output_dir)):
    if npy_file.endswith('.npy') and npy_file in labels:
        img_array = np.load(os.path.join(output_dir, npy_file))
        images.append(img_array)
        image_labels.append(labels[npy_file])


In [8]:
print("Number of images:", len(images))
print("Number of labels:", len(image_labels))

Number of images: 10
Number of labels: 10


In [9]:
# Assuming all images are the same size and reshaped properly for input to a CNN
images = np.array(images).reshape(-1, 300, 300, 1)  # Reshape for CNN, change shape as necessary

# Start constructing the Keras Sequential model.
model = Sequential()

# Add an input layer which is similar to a feed_dict in TensorFlow.
model.add(InputLayer(input_shape=img_shape_full))

# First convolutional layer with ReLU-activation and max-pooling.
model.add(Conv2D(kernel_size=3, strides=1, filters=4, padding='same',
                 activation='relu', name='layer_conv1'))
model.add(MaxPooling2D(pool_size=2, strides=2))

# Second convolutional layer with ReLU-activation and max-pooling.
model.add(Conv2D(kernel_size=5, strides=1, filters=8, padding='same',
                 activation='relu', name='layer_conv2'))
model.add(MaxPooling2D(pool_size=2, strides=2))

# Flatten the output of the convolutional layers
model.add(Flatten())

# First fully-connected layer with ReLU-activation.
model.add(Dense(15, activation='relu'))

# Last fully-connected layer with softmax-activation for use in classification.
model.add(Dense(num_classes, activation='softmax'))

# Compile the model (assuming this is for a classification task)
model.compile(optimizer='adam', loss='BinaryCrossentropy', metrics=['accuracy'])

2024-05-11 16:13:16.484094: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-11 16:13:16.503973: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-11 16:13:16.504086: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [10]:
# Convert lists to numpy arrays
images = np.array(images).reshape(-1, 300, 300, 1)  # Adjust the shape according to your CNN input
image_labels = np.array(image_labels)

# Convert labels to one-hot encoding
image_labels = to_categorical(image_labels)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, image_labels, test_size=0.2, random_state=42)

In [11]:
model.fit(
    x=X_train,
    y=y_train,
    batch_size=2,
    epochs=5,
    verbose="auto",
    callbacks=None,
    validation_split=0.0,
    validation_data=None,
    shuffle=True,
    class_weight=None,
    sample_weight=None,
    initial_epoch=0,
    steps_per_epoch=None,
    validation_steps=None,
    validation_batch_size=None,
    validation_freq=1,
)

model.save('../models/model.h5')

Epoch 1/5


2024-05-11 16:13:17.007321: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
2024-05-11 16:13:17.314081: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f775364b860 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-05-11 16:13:17.314100: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3080 Ti, Compute Capability 8.6
2024-05-11 16:13:17.316981: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1715436797.361232  131929 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


4/4 [==============================] - 1s 3ms/step - loss: 97.3919 - accuracy: 0.7500
Epoch 2/5
4/4 [==============================] - 0s 2ms/step - loss: 134.9989 - accuracy: 0.7500
Epoch 3/5
4/4 [==============================] - 0s 2ms/step - loss: 18.8342 - accuracy: 0.8750
Epoch 4/5
4/4 [==============================] - 0s 2ms/step - loss: 0.2094 - accuracy: 1.0000
Epoch 5/5
4/4 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 1.0000


/home/ahamboeck/anaconda3/envs/open-cv-keras/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
